In [ ]:
import os
import random
import rasterio
import csv
import re



In [ ]:
# Input folder containing subfolders
input_folder = "./data/satellite/original/"  # Relative path to the input folder
output_folder = "./points"  # Relative path to the output folder
os.makedirs(output_folder, exist_ok=True)

# Traverse each subfolder
for subfolder in os.listdir(input_folder):
    subfolder_path = os.path.join(input_folder, subfolder)
    if os.path.isdir(subfolder_path):
        # List all .tif files in the subfolder
        tif_files = [f for f in os.listdir(subfolder_path) if f.endswith('.tif')]
        
        if tif_files:
            # Randomly pick a .tif file
            selected_file = random.choice(tif_files)
            raster_path = os.path.join(subfolder_path, selected_file)
            print(f"Processing {raster_path}")
            
            # Open the raster and extract points
            with rasterio.open(raster_path) as src:
                # Get raster dimensions and transform
                transform = src.transform
                rows, cols = src.height, src.width
                
                # Create a TXT file for output
                output_file = os.path.join(output_folder, f"{os.path.splitext(selected_file)[0]}_grid.txt")
                with open(output_file, mode='w', newline='') as txt_file:
                    writer = csv.writer(txt_file, delimiter=',')
                    # Write header
                    writer.writerow(["X", "Y"])
                    
                    # Write pixel coordinates
                    for row in range(rows):
                        for col in range(cols):
                            x, y = rasterio.transform.xy(transform, row, col, offset="center")
                            writer.writerow([x, y])
                
                print(f"Saved grid points to {output_file}")

In [ ]:
# Directory containing the TXT files
points_folder = "./points/"

# Loop through all files in the directory
for file_name in os.listdir(points_folder):
    if file_name.endswith('.txt'):  # Only process .txt files
        # Use regex to match and extract the desired part of the filename
        match = re.search(r"(training|testing|validation).*", file_name)
        if match:
            # Extract the part of the filename starting from the matched word
            new_name = match.group(0)
            
            # Full paths for renaming
            old_path = os.path.join(points_folder, file_name)
            new_path = os.path.join(points_folder, new_name)
            
            # Rename the file
            os.rename(old_path, new_path)
            print(f"Renamed: {file_name} -> {new_name}")